<a href="https://colab.research.google.com/github/PerdikosFilippos/Lung-Cancer-Prediction-using-NN-SVC-LR/blob/main/Lung_Cancer_Prediction_using_NN%2C_SVC%2C_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
tf.random.set_seed(3) #Generate random state
from tensorflow import keras
from sklearn import svm

In [179]:
data = pd.read_csv('/content/survey lung cancer.csv')

In [180]:
data

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,F,56,1,1,1,2,2,2,1,1,2,2,2,2,1,YES
305,M,70,2,1,1,1,1,2,2,2,2,2,2,1,2,YES
306,M,58,2,1,1,1,1,1,2,2,2,2,1,1,2,YES
307,M,67,2,1,2,1,1,2,2,1,2,2,2,1,2,YES


In [181]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

In [182]:
#Missing Values Check

data.isnull().sum()

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64

In [183]:
data['LUNG_CANCER'].value_counts()

YES    270
NO      39
Name: LUNG_CANCER, dtype: int64

Very unbalanced

In [184]:
#Replace YES/NO with 1/0 and M/F with 1/0
data['LUNG_CANCER'] = data['LUNG_CANCER'].replace({'YES': 1, 'NO': 0})
data['GENDER'] = data['GENDER'].replace({'M': 1, 'F': 0})

In [185]:
data.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,0,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,0,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0


#To address the imbalance (undersampling), first divide the data into cancer and no cancer, so that you are able to make a new dataset with mean values

In [186]:
cancer = data[data.LUNG_CANCER == 1]
no_cancer = data[data.LUNG_CANCER == 0]

In [187]:
no_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 2 to 286
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   GENDER                 39 non-null     int64
 1   AGE                    39 non-null     int64
 2   SMOKING                39 non-null     int64
 3   YELLOW_FINGERS         39 non-null     int64
 4   ANXIETY                39 non-null     int64
 5   PEER_PRESSURE          39 non-null     int64
 6   CHRONIC DISEASE        39 non-null     int64
 7   FATIGUE                39 non-null     int64
 8   ALLERGY                39 non-null     int64
 9   WHEEZING               39 non-null     int64
 10  ALCOHOL CONSUMING      39 non-null     int64
 11  COUGHING               39 non-null     int64
 12  SHORTNESS OF BREATH    39 non-null     int64
 13  SWALLOWING DIFFICULTY  39 non-null     int64
 14  CHEST PAIN             39 non-null     int64
 15  LUNG_CANCER            39 non-null     in

In [188]:
cancer_sample = cancer.sample(n=39)

In [190]:
new_data = pd.concat([cancer_sample, no_cancer], axis=0)

In [191]:
new_data

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
229,1,57,1,1,1,1,2,1,2,1,2,2,2,2,2,1
196,1,64,1,2,2,1,2,1,2,1,2,2,2,1,2,1
119,0,56,1,2,2,2,1,1,2,1,2,1,1,2,2,1
236,1,63,1,2,1,2,1,2,2,2,2,2,2,1,2,1
112,0,68,1,1,1,2,1,2,1,2,1,2,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,0,87,1,1,1,1,2,2,1,1,1,1,2,1,1,0
280,0,59,2,1,1,1,2,2,2,1,1,1,2,1,1,0
281,1,55,2,1,1,1,1,2,2,1,1,1,2,1,2,0
282,1,46,1,2,2,1,1,1,1,1,1,1,1,2,2,0


In [192]:
new_data['LUNG_CANCER'].value_counts()

1    39
0    39
Name: LUNG_CANCER, dtype: int64

In [193]:
new_data.groupby("LUNG_CANCER").mean()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
LUNG_CANCER,,,,,,,,,,,,,,,
0,0.435897,60.743590,1.487179,1.333333,1.307692,1.256410,1.358974,1.487179,1.128205,1.230769,1.179487,1.256410,1.564103,1.128205,1.307692
1,0.512821,62.820513,1.487179,1.641026,1.615385,1.615385,1.538462,1.692308,1.641026,1.692308,1.615385,1.666667,1.666667,1.538462,1.743590


#Split the Data

In [194]:
X = new_data.drop(columns="LUNG_CANCER", axis=1)
X = X.replace({2: 1, 1: 0})
Y = new_data["LUNG_CANCER"]

In [197]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

#Train model using Neural Network

In [198]:
#Setting up layers of neural network
model1 = keras.Sequential([
                          keras.layers.Flatten(input_shape=(15,)), #Flatten converts data into a single-dimension array
                          keras.layers.Dense(8, activation='relu'),
                          keras.layers.Dense(2, activation='sigmoid')#Number of neurons in the output MUST be equal to the number of classes you have. In this case, 2.
])


#Compiling the neural network

model1.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


history = model1.fit(X_train, Y_train, validation_split=0.1, epochs=10)

Epoch 1/10
2/2 [==============================] - 1s 257ms/step - loss: 0.8814 - accuracy: 0.3455 - val_loss: 0.6463 - val_accuracy: 0.5714
Epoch 2/10
2/2 [==============================] - 0s 36ms/step - loss: 0.8827 - accuracy: 0.3091 - val_loss: 0.6235 - val_accuracy: 0.5714
Epoch 3/10
2/2 [==============================] - 0s 53ms/step - loss: 0.8715 - accuracy: 0.3455 - val_loss: 0.6233 - val_accuracy: 0.5714
Epoch 4/10
2/2 [==============================] - 0s 53ms/step - loss: 0.8749 - accuracy: 0.3273 - val_loss: 0.6235 - val_accuracy: 0.5714
Epoch 5/10
2/2 [==============================] - 0s 41ms/step - loss: 0.8689 - accuracy: 0.3455 - val_loss: 0.6086 - val_accuracy: 0.5714
Epoch 6/10
2/2 [==============================] - 0s 44ms/step - loss: 0.8589 - accuracy: 0.3818 - val_loss: 0.6053 - val_accuracy: 0.5714
Epoch 7/10
2/2 [==============================] - 0s 40ms/step - loss: 0.8558 - accuracy: 0.3818 - val_loss: 0.6012 - val_accuracy: 0.5714
Epoch 8/10
2/2 [==========

#Train Using Support Vector Classifier (SVC)

In [210]:
model2 = svm.SVC(kernel="linear")

In [211]:
#training the SVC model with training data
model2.fit(X_train, Y_train)

SVC(kernel='linear')

Model Evaluation on training data

In [212]:
#Accuracy Score on Training Data
X_train_prediction = model2.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,  Y_train)

In [213]:
print("Accuracy on Training Data: ", training_data_accuracy)

Accuracy on Training Data:  0.9516129032258065


Model Evaluation on test data

In [214]:
#Accuracy Score on Test Data
X_test_prediction = model2.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,  Y_test)

In [215]:
print("Accuracy on Test Data: ", test_data_accuracy)

Accuracy on Test Data:  0.875


#Train Using Logistic Regression

In [206]:
model3 = LogisticRegression()

In [216]:
#training the logistic regression model with training data
model3.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation on training data

In [217]:
#Accuracy Score on Training Data
X_train_prediction = model3.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,  Y_train)

In [218]:
print("Accuracy on Training Data: ", training_data_accuracy)

Accuracy on Training Data:  0.967741935483871


Model Evaluation on test data

In [219]:
#Accuracy Score on Test Data
X_test_prediction = model3.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,  Y_test)

In [221]:
print("Accuracy on Test Data: ", training_data_accuracy)

Accuracy on Test Data:  0.967741935483871


#Predictive System using different models

In [227]:
input_data = (0,67,1,1,1,1,1,1,0,0,0,1,1,0,0)

#change input data into a numpy array
input_data_as_numpy_array = np.asarray(input_data)

#reshape array so that the model can predict for a single instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model3.predict(input_data_reshaped) #Change model number accordingly
print(prediction)

if (prediction[0] == 0):
  print("The person does not have lung cancer")
else:
  print("The person has lung cancer")

[1]
The person has lung cancer


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
